In [18]:
# import necessary libraries
import time 
import random
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')


In [19]:
# to see how much time took to web scrap the imdb movies
start_time = datetime.datetime.now()
print('Begin:', start_time)

Begin: 2025-01-31 00:34:52.990337


In [20]:
url = 'https://www.imdb.com/'
path = 'D:/chromedriver-win64/chromedriver.exe'

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection

s = Service(path)

driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get(url)

driver.find_element(By.XPATH, '''//*[@id="suggestion-search-button"]''').click()
time.sleep(2)

element = driver.find_element(By.XPATH, '''//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/div[2]/a[1]''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()

time.sleep(2)
element = driver.find_element(By.XPATH, '''//*[@id="accordion-item-titleTypeAccordion"]/div/section/button[1]''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()

element = driver.find_element(By.XPATH, '''//*[@id="releaseDateAccordion"]/div[1]/label''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()
time.sleep(2)

search1 = driver.find_element(By.CSS_SELECTOR, 'input[class="ipc-textinput__input"][name="release-yearmonth-start-input"]')
actions = ActionChains(driver)
actions.move_to_element(search1).perform()
search1.click()
search1.send_keys(2014)

time.sleep(2)
search2 = driver.find_element(By.CSS_SELECTOR, 'input[class="ipc-textinput__input"][name="release-yearmonth-end-input"]')
search2.click()
search2.send_keys(2024)

driver.find_element(By.XPATH, '''//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[1]/button''').click()
time.sleep(4)

# here i have click the more button for upto 10,000 movies
for click in range(200):
    try:       
        element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button')
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        time.sleep(random.uniform(2, 3)) 
        element.click()
        time.sleep(random.uniform(2, 4))
    except Exception as e:
        print('No more movies. Exist the loop', e)
        break
        
time.sleep(random.uniform(1,2))
content = driver.page_source

time.sleep(random.uniform(1,2))

print('Scrolling stoppped at:', datetime.datetime.now())

Scrolling stoppped at: 2025-01-31 01:16:17.476976


In [21]:
# Here i have extracted 10,000 movie links
soup = BeautifulSoup(content, 'lxml')

containers = soup.find_all( 'div', class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-a69a4297-2 bqNXEn dli-title with-margin")

movie_links = []
for i in containers:
    
    link = i.find('a', class_="ipc-title-link-wrapper")['href']
    movie_links.append(link)

time.sleep(2)
driver.close()
print('Link Extraction stopped at:', datetime.datetime.now())

Link Extraction stopped at: 2025-01-31 01:16:43.122838


In [22]:
# here i have gone through each movie links, extracted the necessary data and appended them on a list
base_url = 'https://www.imdb.com'

details = [['Movie_Name', 'Release_Year', 'Duration', 'Ratings', 'Number_of_Ratings', 'Genres', 'Rated', 
            'Metascore', 'Director', 'Actors', 'User Reviews','Critic_Reviews', 'Description', 'Movie_Image']]

for i in movie_links:
    
    resp = requests.get(base_url + i, headers = {"User-Agent": UserAgent().random})   
    time.sleep(random.uniform(1, 3))
    
    soup = BeautifulSoup(resp.text, 'lxml')    
    box = soup.find('div', class_="sc-9a2a0028-3 bwWOiy")
    
    try:
        name = box.find('h1', class_="sc-ec65ba05-0 dUhRgT").text
    except:
        name = np.nan
    
    try:
        ratings = box.find('span', class_="sc-d541859f-1 imUuxf").text
    except:
        ratings = np.nan
    
    try:
        no_of_ratings = box.find('div', class_="sc-d541859f-3 dwhNqC").text
    except:
        no_of_ratings = np.nan
        
    three_items = box.find_all('li', class_="ipc-inline-list__item")
    try:
        release_yr = three_items[0].text
    except:
        release_yr = np.nan
        
    try:
        duration = three_items[2].text
    except:
        duration = np.nan
        
    try:
        rated = three_items[1].text
    except:
        rated=np.nan
    
    
    box = soup.find('section', class_="sc-70a366cc-4 eMYfBo")
    try:
        gen = box.find_all('span', class_="ipc-chip__text")
        genres = ', '.join([a.text for a in gen])
    except:
        genres = np.nan
        
    try:
        description = box.find('span', class_="sc-42125d72-2 mmImo").text
    except:
        description = np.nan
        
    try:
        director = box.find('a', class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text
    except:
        director = np.nan
    
    
    box = soup.find('li', class_="ipc-metadata-list__item ipc-metadata-list-item--link")
    try:
        act = box.find_all('li', class_="ipc-inline-list__item")
        actors = ', '.join([a.text for a in act])
    except:
        actors = np.nan
        
    try:
        image = soup.find('img', class_="ipc-image")['src']
    except:
        image = np.nan
        
    try:
        metascore = soup.find('span', class_="sc-b0901df4-0 bXIOoL metacritic-score-box").text
    except:
        metascore = np.nan
    
    two_items = soup.find_all('span', class_="score")
    try:
        user_rev = two_items[0].text
    except:
        user_rev = np.nan
        
    try:
        critic_rev = two_items[1].text
    except:
        critic_rev = np.nan
    
    details.append([name, release_yr, duration, ratings, no_of_ratings, genres, rated, 
                    metascore, director, actors, user_rev, critic_rev, description, image])


In [23]:
# created a dataframe of the extracted movies data
df = pd.DataFrame(details[1:], columns = details[0])
df

,Movie_Name,Release_Year,Duration,Ratings,Number_of_Ratings,Genres,Rated,Metascore,Director,Actors,User Reviews,Critic_Reviews,Description,Movie_Image
0,Nosferatu,2024,2h 12m,7.4,123K,"Dark Fantasy, Supernatural Horror, Vampire Hor...",R,78,Robert Eggers,"Lily-Rose Depp, Nicholas Hoult, Bill Skarsgård",1K,348,A gothic tale of obsession between a haunted y...,https://m.media-amazon.com/images/M/MV5BYjU0MT...
1,The Brutalist,2024,3h 34m,8.0,13K,"Epic, Drama",R,90,Brady Corbet,"Adrien Brody, Felicity Jones, Guy Pearce",174,165,When a visionary architect and his wife flee p...,https://m.media-amazon.com/images/M/MV5BM2U0MW...
2,The Substance,2024,2h 21m,7.3,224K,"Body Horror, Dark Comedy, Monster Horror, Psyc...",R,78,Coralie Fargeat,"Demi Moore, Margaret Qualley, Dennis Quaid",1.8K,363,A fading celebrity takes a black-market drug: ...,https://m.media-amazon.com/images/M/MV5BZDQ1NG...
3,Emilia Pérez,2024,2h 12m,5.9,48K,"Pop Musical, Comedy, Crime, Drama, Musical, Th...",R,71,Jacques Audiard,"Jacques Audiard, Thomas Bidegain, Léa Mysius",417,213,Emilia Pérez follows four remarkable women in ...,https://m.media-amazon.com/images/M/MV5BNjBhMW...
4,Gladiator II,2024,2h 28m,6.6,184K,"Action Epic, Adventure Epic, Epic, Period Dram...",R,64,Ridley Scott,"David Franzoni, Peter Craig, David Scarpa",1.8K,278,After his home is conquered by the tyrannical ...,https://m.media-amazon.com/images/M/MV5BZGJhN2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,2023,NaN,7.2,668,"Comedy, Documentary",1h 14m,NaN,Manny Rodriguez,,16,4,Morgan showcases her life via intimate tales a...,https://m.media-amazon.com/images/M/MV5BNmVkMz...
9996,Mafia Inc,2019,NaN,6.8,1.7K,NaN,2h 23m,64,NaN,"Sergio Castellitto, Marc-André Grondin, Gilber...",24,13,NaN,https://m.media-amazon.com/images/M/MV5BNTRkZj...
9997,Pound of Flesh,2015,1h 44m,5.1,7.2K,"Action, Thriller",R,41,Ernie Barbarash,"Jean-Claude Van Damme, John Ralston, Aki Aleong",51,44,A man's heroic attempt to help a woman in dist...,https://m.media-amazon.com/images/M/MV5BMjI2OD...
9998,NaN,2014,1h 50m,6.0,445,"Drama, Family, Music",PG,NaN,Andy Lauer,"David Kauffman, Steven Sessions",6,3,"A singer and songwriter, who gave up his dream...",https://m.media-amazon.com/images/M/MV5BMGY5MT...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Movie_Name         8594 non-null   object
 1   Release_Year       10000 non-null  object
 2   Duration           6719 non-null   object
 3   Ratings            9970 non-null   object
 4   Number_of_Ratings  9970 non-null   object
 5   Genres             8286 non-null   object
 6   Rated              9875 non-null   object
 7   Metascore          4920 non-null   object
 8   Director           8286 non-null   object
 9   Actors             10000 non-null  object
 10  User Reviews       9736 non-null   object
 11  Critic_Reviews     9077 non-null   object
 12  Description        8286 non-null   object
 13  Movie_Image        10000 non-null  object
dtypes: object(14)
memory usage: 1.1+ MB


In [25]:

end_time = datetime.datetime.now()
print('End:', end_time)

total_time = end_time - start_time
print('Total time:', total_time)

End: 2025-01-31 15:23:43.023580
Total time: 14:48:50.033243


In [26]:
# saved the data frame in a csv file so we can impute the missing values into our original dataset.
df.to_csv('imdb.csv', index = False)